<a href="https://colab.research.google.com/github/xinhe-zhang/heatwave/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
import sklearn.ensemble
import scipy.stats
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

!pip install cartopy
import cartopy.crs as ccrs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 77.2 MB/s eta 0:00:00


In [3]:
#Download ERA5 Tmax, 1979-2022, land only, 144*73
!gdown https://drive.google.com/u/1/uc?id=14okWeTrUH1nHZcgbmBZupRM6s5EcVoYu&export=download

Downloading...
From: https://drive.google.com/u/1/uc?id=14okWeTrUH1nHZcgbmBZupRM6s5EcVoYu
To: /content/ERA5_SFC_mx2t_1979-2022_r144x73_day.land.nc
100% 676M/676M [00:03<00:00, 179MB/s]


In [4]:
filename = 'ERA5_SFC_mx2t_1979-2022_r144x73_day.land.nc'
ds = xr.open_dataset(filename)

In [5]:
print(ds)

<xarray.Dataset>
Dimensions:  (time: 16071, lon: 144, lat: 73)
Coordinates:
  * time     (time) int32 19790101 19790102 19790103 ... 20221230 20221231
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 82.5 85.0 87.5 90.0
Data variables:
    mx2t     (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.1.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Tue Sep 05 13:41:31 2023: cdo chname,t2m,mx2t ERA5_SFC_mx2t...
    CDO:          Climate Data Operators version 2.1.1 (https://mpimet.mpg.de...
